In [1]:
initial_spaces = [['w']*8, ['w']*8, ['']*8, ['']*8, ['']*8, ['']*8, ['b']*8, ['b']*8]
empty_board = [['']*8 for _ in range(8)] 
#empty_board = [[0]*8]*8
        
class chessboard(object):
    
    def __init__(self, spaces = initial_spaces):
        #white pieces and black pieces will be dictionaries with keys being board tiles and values being pieces in those tiles
        for i in range(len(spaces)):
            for j in range(len(spaces)):
                empty_board[j][i] = spaces[i][j] #want the board to be properly oriented
        self.spaces = empty_board
        
    def isOccupied(self, position, direction):
        x, y = position
        new_x, new_y = x + direction[0], y + direction[1]
        return bool(board.spaces[new_x][new_y])
    
board = chessboard()

In [2]:
def kill_piece(position):
    pieces[position].alive = False

In [3]:
#abstract class with methods common to all pieces
class chessPiece(object):
    
    def __init__(self, position, is_white, alive = True): #position must be a tuple
        self.position = position
        self.alive = alive
        self.is_white = is_white
        
    def move(self, direction):
        if not self.alive:
            print('That piece has already been taken. Choose a different piece!')
        x, y = self.position
        new_x, new_y = x + direction[0], y + direction[1]
        if any([new_x < 0, new_x > 7, new_y < 0, new_y > 7]):
            print('Not a valid move. That would place your piece off the board!')
        elif (board.spaces[new_x][new_y] == 'w' and self.is_white) or (board.spaces[new_x][new_y] == 'b' and not self.is_white):
            print('Not a valid move. That space is occupied by your own piece!')
        else:
            return (new_x, new_y)
    def is_attacking(self, direction):
        new_pos = self.move(direction)
        if pieces[new_pos] != '':
            if board.isOccupied(self.position, direction) and pieces[new_pos].is_white != self.is_white:
                return True
            else:
                return False
        else:
            return False
        
    def do_move(self, direction):
        x , y = self.position
        new_x, new_y = x + direction[0], y + direction[1]
        new_pos = (new_x, new_y)
        if self.is_white:
                board.spaces[new_x][new_y] = 'w'
                board.spaces[x][y] = ''
        else:
            board.spaces[new_x][new_y] = 'b'
            board.spaces[x][y] = ''
        return new_pos

In [4]:
class whitePawn(chessPiece):
    valid_moves = {(0, 1), (0, 2), (1, 1), (-1, 1)}
    def __init__(self, position, is_white = True, alive = True, has_moved = False):
        super().__init__(position, is_white, alive)
        self.has_moved = has_moved
        
    def whitePawnMove(self, direction):
        x, y = self.position #gonna need this at the last step
        new_x, new_y= x + direction[0], y + direction[1]
        new_pos = super().move(direction) 
        if direction not in self.valid_moves:
            print('That is not a valid move for a pawn! Choose a different move.')
        elif direction in {(1, 1), (-1, 1)} and not board.isOccupied(self.position, direction):
            print('There\'s no black piece on that tile. You can\'t attack.')
        #note that the .move() method in the parent class will detect if any attempted move would place the pawn
        #on a tile occupied by another white piece
        elif direction == (0, 2) and self.has_moved:
            print('This pawn has already moved.')
        elif direction in {(0, 1), (0, 2)} and board.isOccupied(self.position, direction):
            print('That space is already occupied.')
        else:
            if self.is_attacking(direction):
                kill_piece(new_pos)
                pieces[new_pos] = pieces[self.position]
                pieces[self.position] = ''
            self.position = self.do_move(direction) 

In [5]:
class blackPawn(chessPiece):
    valid_moves = {(0, -1), (0, -2), (1, -1), (-1, -1)}
    def __init__(self, position, is_white = False, alive = True, has_moved = False):
        super().__init__(position, is_white, alive)
        self.has_moved = has_moved
        
    def blackPawnMove(self, direction):
        x, y = self.position #gonna need this at the last step
        new_x, new_y= x + direction[0], y + direction[1]
        new_pos = super().move(direction) 
        if direction not in self.valid_moves:
            print('That is not a valid move for a pawn! Choose a different move.')
        elif direction in {(1, -1), (-1, -1)} and not board.isOccupied(self.position, direction):
            print('There\'s no white piece on that tile. You can\'t attack.')
        #note that the .move() method in the parent class will detect if any attempted move would place the pawn
        #on a tile occupied by another black piece
        elif direction == (0, -2) and not self.has_moved:
            print('This pawn has already moved.')
        elif direction in {(0, -1), (0, -2)} and board.isOccupied(self.position, direction):
            print('That space is already occupied.')
        else:
            if self.is_attacking(direction):
                kill_piece(new_pos)
                pieces[new_pos] = pieces[self.position]
                pieces[self.position] = ''
            self.position = self.do_move(direction) 

In [6]:
class king(chessPiece):
    valid_moves = {(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)}
    def __init__(self, position, is_white, alive = True):
        super().__init__(position, is_white, alive)
        
    def is_in_check(self):
        return False                      #placeholder
    
    def kingMove(self, direction):
        x, y = self.position
        new_x, new_y = x + direction[0], y + direction[1]
        new_pos = super().move(direction) 
        if direction not in self.valid_moves:
            print('Not a valid move for a king.')
        elif king(new_pos, self.is_white, alive = True).is_in_check():
            print('That move would put you in check!')
        else:
            if self.is_attacking(direction):
                kill_piece(new_pos)
                pieces[new_pos] = pieces[self.position]
                pieces[self.position] = ''
            self.position = self.do_move(direction)  

In [7]:
class knight(chessPiece):
    valid_moves = {(1, 2), (2, 1), (-1, 2), (2, -1), (1, -2), (-2, 1), (-1, -2), (-2, -1)}
    def __init__(self, position, is_white, alive = True):
        super().__init__(position, is_white, alive)
        
    def knightMove(self, direction):
        x, y = position
        new_x, new_y = x + direction[0], y + direction[1]
        new_pos = super().move(direction) 
        if direction not in self.valid_moves:
            print('Not a valid move for a knight')
        else:
            if self.is_attacking(direction):
                kill_piece(new_pos)
                pieces[new_pos] = pieces[self.position]
                pieces[self.position] = ''
            self.position = self.do_move(direction)  

In [8]:
class bishop(chessPiece):
    valid_moves = [(a, a) for a in range(1, 8)] + [(-b, -b) for b in range(1, 8)] + [(-c, c) for c in range(1, 8)] + [(d, -d) for d in range(1, 8)]
    
    def __init__(self, position, is_white, alive = True):
        super().__init__(position, is_white, alive)
        
    def bishopMove(self, direction):
        new_pos = super().move(direction)
        if direction not in self.valid_moves:
            print('Not a valid move for a bishop')
        else:
            if self.is_attacking(direction):
                kill_piece(new_pos)
                pieces[new_pos] = pieces[self.position]
                pieces[self.position] = ''
            self.position = self.do_move(direction)  

In [9]:
class rook(chessPiece):
    valid_moves = [(0, a) for a in range(1, 8)] + [(a, 0) for a in range(1, 8)] + [(0, -a) for a in range(1, 8)] + [(-a, 0) for a in range(1, 8)]
    
    def __init__(self, position, is_white, alive = True):
        super().__init__(position, is_white, alive)
        
    def rookMove(self, direction):
        new_pos = super().move(direction)
        if direction not in self.valid_moves:
            print('Not a valid move for a rook')
        else:
            if self.is_attacking(direction):
                kill_piece(new_pos)
                pieces[new_pos] = pieces[self.position]
                pieces[self.position] = ''
            self.position = self.do_move(direction)

In [14]:
class queen(chessPiece):
    valid_moves1 = [(a, a) for a in range(1, 8)] + [(-b, -b) for b in range(1, 8)] + [(-c, c) for c in range(1, 8)] + [(d, -d) for d in range(1, 8)]
    valid_moves2 = [(0, a) for a in range(1, 8)] + [(a, 0) for a in range(1, 8)] + [(0, -a) for a in range(1, 8)] + [(-a, 0) for a in range(1, 8)]
    valid_moves = valid_moves1 + valid_moves2
    
    def __init__(self, position, is_white, alive = True):
        super().__init__(position, is_white, alive)
        
    def queenMove(self, direction):
        new_pos = super().move(direction)
        if direction not in self.valid_moves:
            print('Not a valid move for a queen')
        else:
            if self.is_attacking(direction):
                kill_piece(new_pos)
                pieces[new_pos] = pieces[self.position]
                pieces[self.position] = ''
            self.position = self.do_move(direction) 

In [11]:
pieces = dict()

#white pieces
pieces[(0, 0)] = rook((0, 0), is_white = True)
pieces[(7, 0)] = rook((7, 0), is_white = True)
pieces[(1, 0)] = knight((1, 0), is_white = True)
pieces[(6, 0)] = knight((6, 0), is_white = True)
pieces[(2, 0)] = bishop((2, 0), is_white = True)
pieces[(5, 0)] = bishop((5, 0), is_white = True)
pieces[(3, 0)] = queen((3, 0), is_white = True)

for i in range(8):
    pieces[(i, 1)] = whitePawn((i, 1))
    
#empty spaces
for j in range(2, 6):
    for i in range(8):
        pieces[(i, j)] = ''
    
#black pieces
pieces[(0, 7)] = rook((0, 0), is_white = False)
pieces[(7, 7)] = rook((7, 0), is_white = False)
pieces[(1, 7)] = knight((1, 0), is_white = False)
pieces[(6, 7)] = knight((6, 0), is_white = False)
pieces[(2, 7)] = bishop((2, 0), is_white = False)
pieces[(5, 7)] = bishop((5, 0), is_white = False)
pieces[(4, 7)] = queen((3, 0), is_white = False)


for i in range(8):
    pieces[(i, 6)] = blackPawn((i, 6))
    
white_king = king((4, 0), is_white = True)
black_king = king((3, 7), is_white = False)

pieces[(4, 0)] = white_king
pieces[(3, 7)] = black_king

In [15]:
board = chessboard()
pieces[(1, 1)] = queen((1, 1), is_white = True)
pieces[(2, 2)] = rook((2, 2), is_white = False)
board.spaces[2][2] = 'b'
wq1 = pieces[(1, 1)]
br1 = pieces[(2, 2)]
wq1.queenMove((1, 1))
print(pieces[1, 1])
print(pieces[2, 2])
br1.alive

False

In [16]:
br1.rookMove((1, 0))

That piece has already been taken. Choose a different piece!
That piece has already been taken. Choose a different piece!
